In [ ]:
!pip install DeepFace


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from retinaface import RetinaFace
import cv2
import os
import numpy as np

# Function to calculate brightness
def calculate_brightness(img):
    return np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:,:,2])

# Function to calculate contrast
def calculate_contrast(img):
    return img.std()

# Function to calculate noise
def calculate_noise(img):
    return np.var(img)

# Function to calculate sharpness using Laplacian
def calculate_sharpness(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()
def calculate_blur(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# Function to calculate the alignment of the face
def calculate_alignment(landmarks):
    left_eye = np.array(landmarks['left_eye'])
    right_eye = np.array(landmarks['right_eye'])
    delta_y = right_eye[1] - left_eye[1]
    delta_x = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(delta_y, delta_x))
    return abs(angle)
def check_image_size(img, min_size):
    height, width = img.shape[:2]
    return width >= min_size['width'] and height >= min_size['height']

# Function to check if the eyes, nose, and ears are parallel
def are_features_parallel(face_info):
    landmarks = face_info['landmarks']
    eye_left = landmarks['left_eye']
    eye_right = landmarks['right_eye']
    nose = landmarks['nose']
    ear_left = landmarks.get('left_ear', None)
    ear_right = landmarks.get('right_ear', None)

    # Check if eyes are parallel (difference in y-coordinates should be minimal)
    eyes_parallel = abs(eye_left[1] - eye_right[1]) < 20  # Increased tolerance

    # Check if nose is centered between the eyes
    nose_centered = abs(nose[0] - (eye_left[0] + eye_right[0]) / 2) < 20  # Increased tolerance

    # Check if ears are parallel (optional, depends on availability)
    ears_parallel = True
    if ear_left is not None and ear_right is not None:
        ears_parallel = abs(ear_left[1] - ear_right[1]) < 20  # Increased tolerance

    return eyes_parallel and nose_centered and ears_parallel


def quality_assessment_check(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.detect_faces(img)

    for face_id, face_info in faces.items():
        x1, y1, x2, y2 = face_info['facial_area']
        face_img = img[y1:y2, x1:x2]

        # Calculate features
        brightness = calculate_brightness(face_img)
        contrast = calculate_contrast(face_img)
        noise = calculate_noise(face_img)
        sharpness = calculate_sharpness(face_img)
        blur = calculate_blur(face_img)
        alignment = calculate_alignment(face_info['landmarks'])

        # print(f'Brightness: {brightness}, Contrast: {contrast}, Noise: {noise}, Sharpness: {sharpness}')
        # Define threshold values
        threshold_values = {
            'brightness': 100,
            'contrast': 60,
            'noise': 3500,
            'sharpness': 150,
            'blur': 150,# Adjusted based on the Laplacian variance scale
            'alignment': 180
        }
        min_size = {
            'width': 1000,
            'height': 2000
        }
        # Apply thresholds
        # if brightness > 50 and contrast < 80 and noise < 5000 and sharpness < 1800:
        # if brightness > 90 and contrast < 80 and noise < 5000 and sharpness < 150:
            # img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # filename = f'{person_folder}/photo_{accepted_images + 1}.jpg'
            # cv2.imwrite(filename, img)
            # print(f'quality_assessment_check passed.'
        if (brightness > threshold_values['brightness'] and
                        contrast < threshold_values['contrast'] and
                        noise < threshold_values['noise'] and
                        sharpness > threshold_values['sharpness'] and
                        blur < threshold_values['blur']and
                        alignment < threshold_values['alignment']):
            check = True
        else:
            # print('quality_assessment_check failed.')
            check = False

    return face_img, check

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import Dataset, DataLoader
from deepface import DeepFace
import numpy as np
import os

# Define the LSTM model with output dimension 128
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, lengths):
        # Pack the sequence
        packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, _) = self.lstm(packed_input)
        # We only need the last hidden state
        out = self.fc(hn[-1])
        return out

# Model parameters
input_dim = 512  # Dimension of the Facenet512 embeddings
hidden_dim = 256
output_dim = 128  # Output dimension of the LSTM
num_layers = 4  # Number of LSTM layers

# Instantiate the model
model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# import torch
# import numpy as np
# import os
# import cv2
# from deepface import DeepFace
# import matplotlib.pyplot as plt
# from collections import defaultdict  # Add this import

# # Block 1: Code for Loading Class Embeddings and Rounding Off
# lstm_no_aug_path = "/content/drive/MyDrive/emb_Data/lstm_no_prep.pth"
# model.load_state_dict(torch.load(lstm_no_aug_path))
# model.eval()
# #check
# def normalize_brightness_contrast_grayscale(image):
#     # Normalize brightness using LAB color space
#     lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     l = cv2.equalizeHist(l)
#     lab = cv2.merge((l, a, b))
#     normalized_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

#     # Contrast normalization using CLAHE
#     lab = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#     l = clahe.apply(l)
#     lab = cv2.merge((l, a, b))
#     normalized_contrast_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

#     # Convert to grayscale
#     gray_image = cv2.cvtColor(normalized_contrast_image, cv2.COLOR_BGR2GRAY)

#     return gray_image



# # Function to extract features and normalize all images in a folder
# def process_class_images(class_folder):
#     img_paths = [os.path.join(class_folder, img) for img in os.listdir(class_folder)]
#     brightness_values = []
#     contrast_values = []

#     # Calculate brightness and contrast for each image
#     for img_path in img_paths:
#         image = cv2.imread(img_path)
#         brightness, contrast = calculate_image_features(image)
#         brightness_values.append(brightness)
#         contrast_values.append(contrast)

#     # Get average brightness and contrast for the class
#     avg_brightness = np.mean(brightness_values)
#     avg_contrast = np.mean(contrast_values)

#     print(f"Class {class_folder}: Avg Brightness = {avg_brightness}, Avg Contrast = {avg_contrast}")

#     processed_images = []
#     for img_path in img_paths:
#         image = cv2.imread(img_path)
#         # Normalize using average values of brightness and contrast
#         processed_image = normalize_with_custom_values(image, avg_brightness, avg_contrast)
#         processed_images.append(processed_image)

#     return processed_images

# # Block 2: Code for Getting Image Embedding
# def get_initial_image_embedding(image):
#     # if type(image_path) == "str":
#     #     image = cv2.imread(image_path)
#     # else:
#     #     image = image_path

#     if len(image.shape) == 2:
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     processed_image = normalize_brightness_contrast_grayscale(image)
#     if len(processed_image.shape) == 2:
#         processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB)
#     embedding = DeepFace.represent(img_path=processed_image, model_name="Facenet512", detector_backend='skip')
#     embarr = np.array(embedding[0]["embedding"])
#     return torch.tensor(embarr, dtype=torch.float32).unsqueeze(0).unsqueeze(1)

# def get_final_image_embedding(image_path, model):
#     initial_embedding = get_initial_image_embedding(image_path)
#     lengths = torch.tensor([initial_embedding.shape[1]])  # Sequence length is 1
#     with torch.no_grad():
#         final_embedding = model(initial_embedding, lengths)
#     return final_embedding

# # Block 3: Code for Comparing New Image Embedding
# def load_all_embeddings(save_dir):
#     all_embeddings = defaultdict(list)
#     for file in os.listdir(save_dir):
#         if file.endswith('.npy'):
#             class_num = int(file.split('_')[1])
#             vector = np.load(os.path.join(save_dir, file))
#             all_embeddings[class_num].append(torch.tensor(vector, dtype=torch.float32))
#     return all_embeddings

# def compare_with_tolerance(embedding1, embedding2, tolerance=0.00025):
#     return torch.sum(torch.abs(embedding1 - embedding2) <= tolerance).item()

# def result(image_path, model, save_dir=None, tolerance=0.00025):
#     new_image_final_embedding = get_final_image_embedding(image_path, model)
#     # all_embeddings = load_all_embeddings(save_dir)

#     new_image_final_embedding_rounded = torch.round(new_image_final_embedding)

#     # print(f"New Image Embedding:\n{new_image_final_embedding_rounded.squeeze().numpy()}")
#     return new_image_final_embedding_rounded.squeeze().numpy()

# # ------
# # This function will compare each image with every other image in the folder
# def matching_precentage(lis, tolerance=0.1):
#     final_lst = []
#     for start in range(len(lis)):
#         for end in range(len(lis[start:-1])):
#             # print(start, start+end+1)
#             emb = np.sum(abs(lis[start] - lis[start+end+1]) <= tolerance).item()
#             # print(emb)
#             final_lst.append(emb)

#     count = 0
#     for i in final_lst:
#         if i == 128:
#             count += 1

#     if len(final_lst) == 0:
#         matching_percentage = None
#     else:   matching_percentage = int((count/len(final_lst))*100)

#     print(f"Matching Percentage: {matching_percentage}%")
#     print(f"Total Matching Images: {count} out of {len(final_lst)} pairs")
#     return matching_percentage, final_lst


In [ ]:
# from tqdm import tqdm
# import glob

# # dir = "/Users/kartikkhandelwal/IITH/IITH-Soulverse/Latest/Dataset/Ankit"
# # parent_dir = "/Users/kartikkhandelwal/IITH/IITH-Soulverse/Latest/Dataset"
# parent_dir = "/content/drive/MyDrive/New test"

# for dir in glob.glob(parent_dir + "/*"):
#     lis = []
#     qac_img = 0
#     name = dir.split("/")[-1]
#     print(name)
#     # for i in tqdm(range(len(os.listdir(dir)) - 1)):
#     try:
#         for new_image_path in glob.glob(dir + "/*"):
#             cropped_img, check = quality_assessment_check(new_image_path)
#             if check:
#                 qac_img = qac_img + 1
#                 emb = result(cropped_img, model)
#                 lis.append(emb)

#         per, matching = matching_precentage(lis)
#         print(f"Quality Assessment Check Passed: {qac_img} out of {len(os.listdir(dir))}")
#         print(matching)
#         print("-"*25)
#     except Exception as e:
#         print(e)
#         print(new_image_path)
#         print("-"*25)

--------

In [ ]:
import torch
import numpy as np
import os
import cv2
from deepface import DeepFace
from collections import defaultdict
from tqdm import tqdm
import glob

# Load LSTM model weights
def load_model_weights(model, weight_path):
    model.load_state_dict(torch.load(weight_path))
    model.eval()
    return model

# Function to calculate brightness and contrast
def calculate_brightness_contrast(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray_image)
    contrast = gray_image.std()
    return brightness, contrast

# Function to normalize images with average brightness and contrast
def normalize_image(image, avg_brightness, avg_contrast):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    current_brightness, current_contrast = calculate_brightness_contrast(image)

    # Adjust brightness
    brightness_adjustment = avg_brightness - current_brightness
    adjusted_image = gray_image + brightness_adjustment
    adjusted_image = np.clip(adjusted_image, 0, 255)

    # Adjust contrast
    contrast_adjustment = avg_contrast / current_contrast if current_contrast != 0 else 1
    adjusted_image = (adjusted_image - avg_brightness) * contrast_adjustment + avg_brightness
    adjusted_image = np.clip(adjusted_image, 0, 255).astype(np.uint8)

    return cv2.cvtColor(adjusted_image, cv2.COLOR_GRAY2BGR)

# Process images in class folder to calculate average brightness/contrast and normalize them
def process_class_images(class_folder):
    img_paths = [os.path.join(class_folder, img) for img in os.listdir(class_folder)]
    brightness_values = []
    contrast_values = []

    # Calculate brightness and contrast for each image
    for img_path in img_paths:
        image = cv2.imread(img_path)
        brightness, contrast = calculate_brightness_contrast(image)
        brightness_values.append(brightness)
        contrast_values.append(contrast)

    # Average brightness and contrast for the class
    avg_brightness = np.mean(brightness_values)
    avg_contrast = np.mean(contrast_values)
    print(f"Class {class_folder}: Avg Brightness = {avg_brightness}, Avg Contrast = {avg_contrast}")

    processed_images = []
    for img_path in img_paths:
        image = cv2.imread(img_path)
        processed_image = normalize_image(image, avg_brightness, avg_contrast)
        processed_images.append(processed_image)

    return processed_images

# Get embedding from an image using DeepFace
def get_image_embedding(image, model):
    if len(image.shape) == 2:  # Convert grayscale to RGB if necessary
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    embedding = DeepFace.represent(img_path=image, model_name="Facenet512", detector_backend='skip')
    embarr = np.array(embedding[0]["embedding"])
    return torch.tensor(embarr, dtype=torch.float32).unsqueeze(0).unsqueeze(1)

# Extract final image embedding by passing through LSTM model
def get_final_image_embedding(image, model):
    initial_embedding = get_image_embedding(image, model)
    lengths = torch.tensor([initial_embedding.shape[1]])  # Sequence length
    with torch.no_grad():
        final_embedding = model(initial_embedding, lengths)
    return final_embedding

# Load all embeddings from a directory (optional for saving embeddings)
def load_all_embeddings(save_dir):
    all_embeddings = defaultdict(list)
    for file in os.listdir(save_dir):
        if file.endswith('.npy'):
            class_num = int(file.split('_')[1])
            vector = np.load(os.path.join(save_dir, file))
            all_embeddings[class_num].append(torch.tensor(vector, dtype=torch.float32))
    return all_embeddings

# Compare embeddings with a tolerance value
def compare_with_tolerance(embedding1, embedding2, tolerance=0.00025):
    return torch.sum(torch.abs(embedding1 - embedding2) <= tolerance).item()

# Function to get results for image embeddings
def result(image_path, model, tolerance=0.00025):
    new_image_embedding = get_final_image_embedding(image_path, model)
    new_image_embedding_rounded = torch.round(new_image_embedding)
    return new_image_embedding_rounded.squeeze().numpy()

# Calculate matching percentage between embeddings
def matching_percentage(embeddings_list, tolerance=0.1):
    final_lst = []
    for start in range(len(embeddings_list)):
        for end in range(len(embeddings_list[start:-1])):
            emb = np.sum(abs(embeddings_list[start] - embeddings_list[start+end+1]) <= tolerance).item()
            final_lst.append(emb)

    count = 0
    for i in final_lst:
        if i == 128:
            count += 1

    matching_percentage = int((count / len(final_lst)) * 100) if len(final_lst) > 0 else None
    print(f"Matching Percentage: {matching_percentage}%")
    print(f"Total Matching Images: {count} out of {len(final_lst)} pairs")
    return matching_percentage, final_lst

# Main loop for processing images in directories
parent_dir = "/content/drive/MyDrive/New test"

for dir in glob.glob(parent_dir + "/*"):
    embeddings_list = []
    qac_img = 0
    name = dir.split("/")[-1]
    print(name)

    try:
        for image_path in glob.glob(dir + "/*"):
            cropped_img, check = quality_assessment_check(image_path)
            if check:
                qac_img += 1
                embedding = result(cropped_img, model)
                embeddings_list.append(embedding)

        percentage, matching = matching_percentage(embeddings_list)
        print(f"Quality Assessment Check Passed: {qac_img} out of {len(os.listdir(dir))}")
        print(matching)
        print("-" * 25)

    except Exception as e:
        print(e)
        print(image_path)
        print("-" * 25)


Ankit 
Matching Percentage: None%
Total Matching Images: 0 out of 0 pairs
Quality Assessment Check Passed: 0 out of 45
[]
-------------------------
Arpit
Matching Percentage: None%
Total Matching Images: 0 out of 0 pairs
Quality Assessment Check Passed: 0 out of 36
[]
-------------------------
Pabitra_sir
Matching Percentage: None%
Total Matching Images: 0 out of 0 pairs
Quality Assessment Check Passed: 0 out of 29
[]
-------------------------
VISHNU
Matching Percentage: None%
Total Matching Images: 0 out of 0 pairs
Quality Assessment Check Passed: 0 out of 37
[]
-------------------------


In [ ]:
matching

[]

In [ ]:
lis

[]